## UVA MSDS Capstone Project Data Processing Pipeline Step 2
- This notebook marks a shift to using Python for the rest of the analysis
- This notebook takes the files created in Step 1 and creates a large table containing
    - Taxonomy assignments
    - Sequence counts 
    - Sample information
- One assumption used to lessen the memory loads and constraints of the project was using ~6,000 common sequences across files as the sequences of interests. After aggregation across Genus level, this allowed us to have ~800 distinct genera to investigate. This method could be altered in future bioinformatic pipelines, but was sufficient for the task at hand.

In [1]:
import pandas as pd
import numpy as np
import os
import functools

### Listing out the files for looping

In [5]:
# folder path
dir_path = r'150NT_csvs'

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
        
# remove these starting files (used this file for the taxonomy) 
# Will use these two files to find the most common sequences to use as a baseline for our analysis
res.remove("130252.csv")
res.remove("131011.csv")

# confirm starting file is not in the list
print(res)

['127044.csv', '127158.csv', '127159.csv', '127271.csv', '127292.csv', '127307.csv', '127419.csv', '128190.csv', '128225.csv', '128246.csv', '128291.csv', '128310.csv', '128319.csv', '128433.csv', '128483.csv', '128555.csv', '128622.csv', '128663.csv', '128669.csv', '128705.csv', '128764.csv', '128788.csv', '128801.csv', '128817.csv', '128961.csv', '129003.csv', '129020.csv', '129022.csv', '129074.csv', '129085.csv', '129154.csv', '129191.csv', '129318.csv', '129435.csv', '129441.csv', '129487.csv', '129729.csv', '129741.csv', '130045.csv', '130065.csv', '130091.csv', '130336.csv', '130532.csv', '130579.csv', '130691.csv', '131135.csv', '131162.csv', '131201.csv', '131417.csv', '131423.csv', '131433.csv', '131451.csv', '131459.csv', '131495.csv', '131583.csv', '131755.csv', '131762.csv', '131775.csv', '131792.csv', '132036.csv', '132107.csv', '132133.csv', '132142.csv', '132153.csv', '132366.csv', '132390.csv', '132546.csv', '132773.csv', '133199.csv', '133335.csv', '133650.csv', '1337

In [4]:
## Reading in the two largerst feature tables
starter_df = pd.read_csv("150NT_csvs/130252.csv")
starter_df2 = pd.read_csv("150NT_csvs/131011.csv")

## Inner join on the sequences (keeps the common columns)
most_commons = starter_df.merge(starter_df2, on = "Unnamed: 0", how = "inner")

## ~6,087 unique sequences to use as a basis for investigation
most_commons.shape[0]

6087

In [6]:
## Creating a list of all the dataframes to be able to condense into one frame in the next step

## Initiationg a list and appending the first frame created above
dataframes = []
dataframes.append(most_commons)

## Appeneding the rest of the dataframes to the list of dataframes
for i in range(len(res)):
    dataframes.append(pd.read_csv("150NT_csvs/" + res[i]))

In [8]:
## Condensing all of the resultant csvs into one frame
df_final = functools.reduce(lambda left,right: pd.merge(left,right,on=['Unnamed: 0'], how='left'), dataframes)

C:\Users\Capstone1\AppData\Local\Temp\2\ipykernel_14964\3632633422.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'10317.000036921_x', '10317.000049843_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_final = functools.reduce(lambda left,right: pd.merge(left,right,on=['Unnamed: 0'], how='left'), dataframes)


In [9]:
df_final.shape

(6087, 30519)

## Adjusting the condensed dataframe
- Joining the survery information
- Deduplication
- Adjusting orientation of the dataset

In [17]:
## Transposing the condensed dataframe and resetting the index to use the sample id for a join to the 
## sample survery information
big = df_final.set_index("Unnamed: 0").T
big = big.reset_index()

In [12]:
## Reading in the sample survey information
info = pd.read_csv("metadata_files/sample_info.csv")

In [19]:
## joining the sample survery information on the "index" column from the condensed df on the "sample_name" from the survey
join = big.merge(info, left_on = "index", right_on = "sample_name")

In [22]:
## Viewing the diagnosis distribution
join.value_counts("diagnosis")

diagnosis
control           18062
diabetes            453
kidney_disease      288
dtype: int64

In [23]:
## Displaying the joined table with "index" = sample name, sequence counts, and survey information
join

,index,TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGCGCGTAGGCGGTTTTTTAAGTCTGATGTGAAAGCCCACGGCTCAACCGTGGAGGGTCATTGGAAACTGGAAAACTTGAGTGCAGAAGAGGAAAGTGGAATTCCATG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGGGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATTCCGGGGCTTAACTCCGGGCGTGCAGGCGATACGGGCATAACTTGAGTACTGTAGGGGTAACTGGAATTCCTG,TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTACGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTGTGATAGAGGGTGGTAGAATTTCAGG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTTTGTGTAAGCCCGCAGCTTAACTGCGGGACTGCAGGCGATACGGGCATAACTTGAGTGCTGTAGGGGAGACTGGAATTCCTG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGCGGTTTGTCACGTCGTCTGTGAAATCCTAGGGCTTAACCCTGGACGTGCAGGCGATACGGGCTGACTTGAGTACTACAGGGGAGACTGGAATTTCTGG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATCCCGGGGCTTAACTTCGGGCGTGCAGGCGATACGGGCATAACTAGAGTGCTGTAGGGGAGACTGGAATTCCTG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATTCCGGGGCTTAACTCCGGGCGTGCAGGCGATACGGGCATAACTTGAGTGCTGTAGGGGAGACTGGAATTCCTG,TACAGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGAGCGTAGGTGGCTTGATAAGTCAGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATCTGAAACTGTTAGGCTAGAGTAGGTGAGAGGGAAGTAGAATTTCAGG,TACAGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGAGCGTAGGTGGCTTAATAAGTCAGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATCTGATACTGTTGGGCTAGAGTAGGTGAGAGGGAGGTAGAATTTCAGG,...,cdiff,liver_disease,lung_disease,diet_type,whole_grain_frequency,meat_eggs_frequency,milk_cheese_frequency,prepared_meals_frequency,age_cat,diagnosis
0,10317.000041155,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Regularly (3-5 times/week),Occasionally (1-2 times/week),Regularly (3-5 times/week),Daily,60s,control
1,10317.000063283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Occasionally (1-2 times/week),Regularly (3-5 times/week),Daily,Rarely (less than once/week),20s,control
2,10317.000063235,3.0,26.0,0.0,38.0,3.0,92.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Vegan,Occasionally (1-2 times/week),Never,Never,Occasionally (1-2 times/week),20s,control
3,10317.000047501,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Occasionally (1-2 times/week),60s,control
4,10317.000039677,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,I do not have this condition,NaN,I do not have this condition,Omnivore,Rarely (less than once/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Occasionally (1-2 times/week),60s,control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18798,10317.X00215804,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,I do not have this condition,Diagnosed by an alternative medicine practitioner,I do not have this condition,Omnivore,Occasionally (1-2 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Rarely (less than once/week),50s,control
18799,10317.X00215878,0.0,15.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,I do not have this condition,I do not have this condition,I do not have this condition,Vegetarian but eat seafood,Regularly (3-5 times/week),Regularly (3-5 times/week),Daily,Rarely (less than once/week),30s,control
18800,10317.X00215806,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,I do not have this condition,I do not have this condition,"Diagnosed by a medical professional (doctor, p...",Vegetarian,Daily,Rarely (less than once/week),Occasionally (1-2 times/week),Rarely (less than once/week),30s,control
18801,10317.X00215884,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,...,I do not have this condition,I do not have this condition,I do not have this condition,Vegetarian,Regularly (3-5 times/week),Occasionally (1-2 times/week),Regularly (3-5 times/

In [26]:
## Saving this dataframe to output_files folder to avoid rerunning the above code.
join.to_csv("output_files/allseqs_and_sample_info.csv")

## Creating Count Files
- Now that we have the large table with the sample information and taxonomy attached we will create sets of count tables at different taxonomic levels for analysis
- Our project focuses primarily on the Genus level, but code for the other levels are also found in this notebook

In [7]:
## Read the table created above
data = pd.read_csv("output_files/allseqs_and_sample_info.csv")
## Drop unneccesary colums
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

C:\Users\Capstone1\AppData\Local\Temp\2\ipykernel_3628\2927605054.py:2: DtypeWarning: Columns (1,6095,6102,6104,6105) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("output_files/allseqs_and_sample_info.csv")


,index,TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGCGCGTAGGCGGTTTTTTAAGTCTGATGTGAAAGCCCACGGCTCAACCGTGGAGGGTCATTGGAAACTGGAAAACTTGAGTGCAGAAGAGGAAAGTGGAATTCCATG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGGGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATTCCGGGGCTTAACTCCGGGCGTGCAGGCGATACGGGCATAACTTGAGTACTGTAGGGGTAACTGGAATTCCTG,TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTACGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTGTGATAGAGGGTGGTAGAATTTCAGG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTTTGTGTAAGCCCGCAGCTTAACTGCGGGACTGCAGGCGATACGGGCATAACTTGAGTGCTGTAGGGGAGACTGGAATTCCTG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGCGGTTTGTCACGTCGTCTGTGAAATCCTAGGGCTTAACCCTGGACGTGCAGGCGATACGGGCTGACTTGAGTACTACAGGGGAGACTGGAATTTCTGG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATCCCGGGGCTTAACTTCGGGCGTGCAGGCGATACGGGCATAACTAGAGTGCTGTAGGGGAGACTGGAATTCCTG,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATTCCGGGGCTTAACTCCGGGCGTGCAGGCGATACGGGCATAACTTGAGTGCTGTAGGGGAGACTGGAATTCCTG,TACAGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGAGCGTAGGTGGCTTGATAAGTCAGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATCTGAAACTGTTAGGCTAGAGTAGGTGAGAGGGAAGTAGAATTTCAGG,TACAGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGAGCGTAGGTGGCTTAATAAGTCAGATGTGAAATCCCCGGGCTTAACCTGGGAACTGCATCTGATACTGTTGGGCTAGAGTAGGTGAGAGGGAGGTAGAATTTCAGG,...,cdiff,liver_disease,lung_disease,diet_type,whole_grain_frequency,meat_eggs_frequency,milk_cheese_frequency,prepared_meals_frequency,age_cat,diagnosis
0,10317.000041,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Regularly (3-5 times/week),Occasionally (1-2 times/week),Regularly (3-5 times/week),Daily,60s,control
1,10317.000063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Occasionally (1-2 times/week),Regularly (3-5 times/week),Daily,Rarely (less than once/week),20s,control
2,10317.000063,3.0,26.0,0.0,38.0,3.0,92.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Vegan,Occasionally (1-2 times/week),Never,Never,Occasionally (1-2 times/week),20s,control
3,10317.000048,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,I do not have this condition,I do not have this condition,I do not have this condition,Omnivore,Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Occasionally (1-2 times/week),60s,control
4,10317.00004,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,I do not have this condition,NaN,I do not have this condition,Omnivore,Rarely (less than once/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Occasionally (1-2 times/week),60s,control


In [8]:
## Will utilize this table to make count tables
taxa = pd.read_csv("metadata_files/big_tax_table.csv")
taxa.rename({'Unnamed: 0':'seq'}, axis=1, inplace=True)
taxa.head()

,seq,Kingdom,Phylum,Class,Order,Family,Genus
0,TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGCG...,Bacteria,Firmicutes,Bacilli,Bacillales,Staphylococcaceae,Staphylococcus
1,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGGGCT...,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Corynebacterium_1
2,TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTA...,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Pseudoalteromonadaceae,Pseudoalteromonas
3,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCT...,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Corynebacterium_1
4,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCT...,Bacteria,Actinobacteria,Actinobacteria,Corynebacteriales,Corynebacteriaceae,Lawsonella


In [9]:
## Combining the taxonomy data with the AGP data

## dictionaries with mapping of sequences to taxa

phyla = pd.Series(taxa.Phylum.values, taxa.seq.values).to_dict()
clas = pd.Series(taxa.Class.values, taxa.seq.values).to_dict()
order = pd.Series(taxa.Order.values, taxa.seq.values).to_dict()
fam = pd.Series(taxa.Family.values, taxa.seq.values).to_dict()
genus = pd.Series(taxa.Genus.values, taxa.seq.values).to_dict()

In [10]:
# some manipulation before function call

agg_df = data.set_index('index').T.reset_index()
agg_df.rename({'index':'old_cols'}, axis=1, inplace=True)
agg_df.fillna(0)

index,old_cols,10317.000041155,10317.000063283,10317.000063235,10317.000047501,10317.000039677,10317.000039724,10317.000063163,10317.000039726,10317.000039728,...,10317.X00215798,10317.X00215505,10317.X00215863,10317.X00215800,10317.X00215867,10317.X00215804,10317.X00215878,10317.X00215806,10317.X00215884,10317.X00215889
0,TACGTAGGTGGCAAGCGTTATCCGGAATTATTGGGCGTAAAGCGCG...,0.0,0.0,3.0,2.0,4.0,4.0,3.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGGGCT...,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,2.0,0.0,4.0,0.0,0.0,15.0,0.0,0.0,0.0
2,TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTA...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCT...,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCT...,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6110,meat_eggs_frequency,Occasionally (1-2 times/week),Regularly (3-5 times/week),Never,Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Occasionally (1-2 times/week),...,Regularly (3-5 times/week),Daily,Daily,Regularly (3-5 times/week),Daily,Regularly (3-5 times/week),Regularly (3-5 times/week),Rarely (less than once/week),Occasionally (1-2 times/week),Daily
6111,milk_cheese_frequency,Regularly (3-5 times/week),Daily,Never,Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Regularly (3-5 times/week),Daily,...,Occasionally (1-2 times/week),Occasionally (1-2 times/week),Regularly (3-5 times/week),Daily,Regularly (3-5 times/week),Regularly (3-5 times/week),Daily,Occasionally (1-2 times/week),Regularly (3-5 times/week),Occasionally (1-2 times/week)
6112,prepared_meals_frequency,Daily,Rarely (less than once/week),Occasionally (1-2 times/week),Occasionally (1-2 times/week),Occasionally (1-2 times/week),Occasionally (1-2 times/week),Regularly (3-5 times/week),Occasionally (1-2 times/week),Rarely (less than once/week),...,Occasionally (1-2 times/week),Occasionally (1-2 times/week),Occasionally (1-2 times/week),Rarely (less than once/week),Occasionally (1-2 times/week),Rarely (less than once/week),Rarely (less than once/week),Rarely (less than once/week),Occasionally (1-2 times/week),Occasionally (1-2 times/week)
6113,age_cat,60s,20s,20s,60s,60s,50s,20s,50s,60s,...,50s,70+,30s,30s,30s,50s,30s,30s,70+,60s


In [11]:
# grouping is a dict of sequence values to group names
def tax_counts(grouping):
    
    # empty data frame to hold results
    group_counts = pd.DataFrame()
    
    # replaces sequences with grouping names
    
    old_cols = list(agg_df.old_cols)
    replaced_cols = [x if x not in grouping else grouping[x] for x in old_cols]
    group_df = agg_df.copy(deep=True) # maintains integrity of agg_df for later taxa
    group_df['old_cols'] = replaced_cols
    
    # 28 items in this list do not need to be aggregated (survey data)

    relevant = group_df.old_cols.unique()[:-28]
    rest = group_df.old_cols.unique()[-28:]
    
    # gets sum of one group across all observations
    for group in relevant:
        sum_val = group_df[group_df.old_cols == group].set_index('old_cols').T.sum(axis=1)
        temp_df = pd.DataFrame(sum_val)
        temp_df = temp_df.rename({0:group}, axis=1).T  
        group_counts = pd.concat([group_counts, temp_df])
    
    # recombine with survey data
    group_counts.reset_index()
    group_counts.rename({'index':'old_cols'}, axis=1, inplace=True)
    count = group_counts.reset_index().rename({'index':'old_cols'}, axis=1)
    
    # put observations back as rows
    output_df = count.append(group_df.tail(28).reset_index(drop=True)).set_index('old_cols').T
    
    return output_df 

In [12]:
## Calling the above function to create the respective count tables by taxonomy level
phyla_counts = tax_counts(phyla)
class_counts = tax_counts(clas)
order_counts = tax_counts(order)
fam_counts = tax_counts(fam)
genus_counts = tax_counts(genus)

C:\Users\Capstone1\AppData\Local\Temp\2\ipykernel_3628\2599888454.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = count.append(group_df.tail(28).reset_index(drop=True)).set_index('old_cols').T
C:\Users\Capstone1\AppData\Local\Temp\2\ipykernel_3628\2599888454.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = count.append(group_df.tail(28).reset_index(drop=True)).set_index('old_cols').T
C:\Users\Capstone1\AppData\Local\Temp\2\ipykernel_3628\2599888454.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = count.append(group_df.tail(28).reset_index(drop=True)).set_index('old_cols').T
C:\Users\Capstone1\AppData\Local\Temp\2\ipykernel_3628\2599888454.py:32: FutureWarning: The frame.appen

In [13]:
## Saving the outputs to use later
phyla_counts.to_csv('count_files/phyla_counts.csv')
class_counts.to_csv('count_files/class_counts.csv')
order_counts.to_csv('count_files/order_counts.csv')
fam_counts.to_csv('count_files/fam_counts.csv')
genus_counts.to_csv('count_files/genus_counts.csv')

## Summary of Notebook 2
- This notebook condenses all the 150NT CSV files into a single dataframe containing ~6,000 unique sequences
- This notebook assigns values at various taxonomic levels to the condensed AGP 150NT files
- This notebook joins the assigned taxonomy 150NT files with the sample survey information
- This notebook creates aggregated count files by the respective taxonomic level